In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse.linalg import svds

In [2]:
df_order = pd. read_csv(r'C:\Users\LetsGo\Desktop\recommendor\order_Data.csv')
df_order.head()

,Oid,Uid,Pid,TotalPrice,ShippingAddress,PaymentStatus,Status,Date
0,O-1098,U-101,34,16,Paris,Card,Delivered,06/01/19
1,O-1686,U-101,17,300,Paris,Wallet,Delivered,07/02/19
2,O-1727,U-101,38,240,Paris,Card,Delivered,10/02/19
3,O-1840,U-101,19,360,Paris,Card,Delivered,15/02/19
4,O-2045,U-101,1,200,Paris,Card,Delivered,26/02/19


In [3]:
def matrix_factorization_svd(df_order,k_svd):
    
    df_order_count = df_order.groupby(['Uid', 'Pid']).size().reset_index(name='counts')
    user_list = df_order['Uid'].unique().tolist()
    prod_list = df_order['Pid'].unique().tolist()
    userdf = pd.DataFrame({'Uid':user_list})
    proddf = pd.DataFrame({'Pid':prod_list})
    userdf['key'] = 0
    proddf['key'] = 0
    df_merge = userdf.merge(proddf, how='outer')
    res = df_merge.merge(df_order_count, how='left', left_on=['Uid', 'Pid'], right_on=['Uid', 'Pid'])
    res['transaction'] = np.where(res['counts'].isnull(),res['key'],res['counts'])
    res['transaction'] = np.where(res['transaction']==5,4,res['transaction'])
       
    #Creating a sparse pivot table with users in rows and items in columns
    users_items_pivot_matrix_df = res.pivot(index='Uid', columns='Pid',values='transaction').fillna(0)
    
    ### creating user index df
    user_list = users_items_pivot_matrix_df.index.tolist()
    user_dict = dict(enumerate(user_list))
    df_user=pd.DataFrame(list(user_dict.items()),columns=['index_val', 'user_id']) 
    
    ### applying SVD Matrix Factorization algorithm
    R = users_items_pivot_matrix_df.as_matrix()
    user_ratings_mean = np.mean(R, axis = 1)
    R_demeaned = R - user_ratings_mean.reshape(-1, 1)
    U, sigma, Vt = svds(R_demeaned, k = k_svd)
    sigma = np.diag(sigma)
    all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
    
    ### creating final prediction file
    preds_df = pd.DataFrame(all_user_predicted_ratings, columns = users_items_pivot_matrix_df.columns)
    preds_df['index_val'] = preds_df.index
    prediction_final = preds_df.merge(df_user, how='left', left_on=['index_val'], right_on=['index_val'])
    prediction_final = prediction_final.drop('index_val', 1)
    pivot_prediction = prediction_final.melt('user_id', var_name='Pid', value_name='Val')
    pivot_prediction = pivot_prediction.sort_values(by=['user_id','Val'],ascending=[True,False]).reset_index(drop=True)
    pivot_prediction.columns = ['Uid', 'Pid','value']
    pivot_prediction['Pid'] = pivot_prediction.Pid.astype(int)
    prediction_final = pivot_prediction.merge(df_order_count, how='left', left_on=['Uid', 'Pid'], right_on=['Uid', 'Pid'])
    prediction_final['used_before_flag'] = np.where(prediction_final['counts']>=1,1,0)
    
    pivot_prediction.columns = ['Uid', 'Pid','value']
    pivot_prediction['Pid'] = pivot_prediction.Pid.astype(int)
    prediction_final = pivot_prediction.merge(df_order_count, how='left', left_on=['Uid', 'Pid'], right_on=['Uid', 'Pid'])
    prediction_final['used_before_flag'] = np.where(prediction_final['counts']>=1,1,0)
    prediction_final = prediction_final.drop('counts', 1)
    prediction_final['row_count_new'] = prediction_final.groupby(['Uid','used_before_flag']).cumcount()+1
    prediction_final['row_count_overall'] = prediction_final.groupby(['Uid']).cumcount()+1
    prediction_final = prediction_final.rename(columns={"value": "recommendor_value"})
    
    return prediction_final


In [4]:
recommendation_output = matrix_factorization_svd(df_order,15)

C:\Users\LetsGo\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [10]:
recommendation_output.to_csv(r'C:\Users\LetsGo\Desktop\recommendor\recommendation_output.csv', sep='\t', encoding='utf-8')

In [11]:
recommendation_output.head(40)

,Uid,Pid,recommendor_value,used_before_flag,row_count_new,row_count_overall
0,U-101,21,1.977905,1,1,1
1,U-101,1,1.402972,1,2,2
2,U-101,33,1.362835,1,3,3
3,U-101,19,1.235478,1,4,4
4,U-101,6,1.163701,1,5,5
5,U-101,38,1.020554,1,6,6
6,U-101,29,0.953251,1,7,7
7,U-101,22,0.944942,1,8,8
8,U-101,37,0.944884,1,9,9
9,U-101,24,0.933709,1,10,10
